In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import matplotlib.pyplot as plt
import numpy as np
import random
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from utils import *
from siamese import *
from visualize_attributes import *
plt.ioff()

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
root_path = os.path.dirname(os.getcwd())
data_path = os.path.join(root_path, "data/multi_classed_grouped_data")
products = os.listdir(data_path)
train = True
visualize = True
#products = ['bottle', 'hazelnut', 'metal_nut', 'toothbrush', 'transistor', 'leather', 'cable',  'zipper', 'wood']
print(f"Using {device}")

transform = transforms.Compose([transforms.ToTensor(),
                            transforms.Resize((1024,1024), antialias=True)
                            ])

for backbone in ['resnet18', 'resnet34', 'resnet50', 'wide_resnet', 'vgg']:
    print('---------------------------------------------- ' + backbone + ' ----------------------------------------------')
    for product in products:
        print('----------------------------------- ' + product + ' -----------------------------------')
        model_path = os.path.join(root_path, 'models', product + '_siamese_' + backbone + '_subclass_sampling.pth')
        if not os.path.exists(model_path):
            print('No model for this backbone')
            continue
        if backbone in ['wide_resnet', 'vgg11']:
            device = 'cpu'
        print(model_path)
        # Load data dicts
        product_path = os.path.join(data_path, product)
        json_path = os.path.join(root_path, 'data/augmented_train_test_split', product)
        train_dict = json.load(open(os.path.join(json_path, product + '_train_dict.json'), 'r'))
        test_dict = json.load(open(os.path.join(json_path, product + '_test_dict.json'), 'r'))

        # Load model image
        good_images_dict = dict([(image_path, label) for image_path, label in train_dict.items()
                            if label == 1])
        model_image_dict = {list(good_images_dict.items())[0][0] : list(good_images_dict.items())[0][1]}
        
        model_dataset = ProductDataset(model_image_dict, transform=transform)
        model_dataloader = DataLoader(model_dataset, batch_size=1, shuffle=True)

        model_image, _, model_label, _ = next(iter(model_dataloader))
        model_label = model_label[0, 0]
        
        # Load model
        n_classes = len(np.unique(list(train_dict.values())))
        model = SiameseNetwork(n_classes, backbone=backbone)
        
        model.load_state_dict(torch.load(model_path))
        model.eval()
        
        all_score = []
        i = 0
        for image_path in tqdm(list(test_dict.keys())):
            image_dict = {image_path: test_dict[image_path]}
            
            test_dataset = ProductDataset(image_dict, transform=transform)
            test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
            test_image, _, test_label, _ = next(iter(test_dataloader))
            test_label = test_label[0, 0]
            
            all_distance = []
            model.to(device)
            
            # print(model_label, test_label)
            output_1, output_2 = model(model_image.to(device), test_image.to(device))
            euclidean_distance = F.pairwise_distance(output_1, output_2, keepdim=True).item()
            
            all_score.append(np.array([1, int(test_label), 1 == int(test_label), euclidean_distance]))
            # print(avg_distance)

            if i % 5 == 0:
                model_image_np = np.array(model_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                test_image = np.array(test_image[0].permute(1, 2, 0)*255, dtype=np.uint8)
                label_1 = "good"
                label_2 = "defect" if test_label.numpy() == 0 else "good"
                label = "same class" if label_1 == label_2 else "different class"
                fig, ax = plt.subplots(1, 2, figsize=(10, 5))
                ax[0].imshow(model_image_np[:,:,::-1])
                ax[0].set_xlabel(label_1, weight = 'bold', fontsize = 20)
                #ax[0].axis('off')
                ax[1].imshow(test_image[:,:,::-1])
                ax[1].set_xlabel(label_2, weight = 'bold', fontsize = 20)
                #ax[1].axis('off')
                fig.suptitle('Dissimilarity score: ' + str(euclidean_distance))
                figure_name = product + '_siamese_' + backbone + '_visualize_' + str(i) + '_one_good_image.png'
                figure_path = os.path.join(root_path, 
                        'Figure/visualize_inference/one_good_image', product)
                if not os.path.exists(figure_path):
                    os.mkdir(figure_path)
                figure_path = os.path.join(figure_path, figure_name)
                fig.savefig(figure_path)
            i += 1
        result = np.array(all_score)
        score_path = os.path.join(root_path, 'result/similarity_scores', product + '_siamese_' + backbone + '_one_good_image.csv')
        np.savetxt(score_path, result, delimiter=",")
        
        same_class = result[:,2]
        if len(np.unique(same_class)) != 2:
            print(np.unique(same_class))
        similarity_scores = result[:,3]
        
        dissimillar_idx = np.where(same_class == 0.)
        dissimillar_score = similarity_scores[dissimillar_idx]
        simillar_idx = np.where(same_class == 1.)
        simillar_score = similarity_scores[simillar_idx]
        #print(dissimillar_idx, simillar_idx)
        #print(dissimillar_score, simillar_score)
        fig, ax = plt.subplots(1, 2, figsize=(16, 8))
        ax[0].hist(simillar_score, edgecolor='black',)
                   #bins=np.arange(simillar_score.min(), simillar_score.max(), 0.1))
        ax[0].set_xlabel('Distance', fontsize = 10)
        ax[0].set_ylabel('Number of instance', fontsize = 10)
        ax[0].set_title(product + ': Similarity score distribution for images for GOOD class')
        ax[1].hist(dissimillar_score, edgecolor='black',)
                   #bins=np.arange(dissimillar_score.min(),dissimillar_score.max(), 0.1))
        ax[1].set_xlabel('Distance', fontsize = 10)
        ax[1].set_ylabel('Number of instance', fontsize = 10)
        ax[1].set_title(product + ': Similarity score distribution for images for DEFECT class')
        fig.show()

        figure_name = product + '_siamese_' + backbone + '_similarity_score_distribution' + '_one_good_image.png'
        figure_path = os.path.join(root_path, 
                'Figure/one_good_image_similarity_score_histogram/' + figure_name)
        fig.savefig(figure_path)


        # fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        # ax.hist(simillar_score, edgecolor='black', label = 'good')
        # ax.hist(dissimillar_score, alpha=0.4, edgecolor='black', label = 'defect')
        #            #bins=np.arange(dissimillar_score.min(),dissimillar_score.max(), 0.1))
        # ax.set_xlabel('Distance', fontsize = 10)
        # ax.set_ylabel('Number of instance', fontsize = 10)
        # ax.set_title(product + ': Similarity score distribution of both GOOD and DEFECT')
        # ax.legend()
        # figure_name = product + '_siamese_' + backbone + 'all_similarity_score_distribution' + '_embedded_scoring.png'
        # figure_path = os.path.join(root_path, 
        #         'Figure/embedded_similarity_score_histogram/' + figure_name)
        # fig.savefig(figure_path)

Using cuda
---------------------------------------------- resnet18 ----------------------------------------------
----------------------------------- bottle -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/bottle_siamese_resnet18_subclass_sampling.pth


 92%|█████████▏| 100/109 [00:13<00:00, 10.64it/s]/tmp/ipykernel_11921/1943467622.py:74: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(1, 2, figsize=(10, 5))
100%|██████████| 109/109 [00:14<00:00,  7.45it/s]
/tmp/ipykernel_11921/1943467622.py:116: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()


----------------------------------- cable -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/cable_siamese_resnet18_subclass_sampling.pth


100%|██████████| 149/149 [00:16<00:00,  8.81it/s]


----------------------------------- capsule -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/capsule_siamese_resnet18_subclass_sampling.pth


...

----------------------------------- cable -----------------------------------
/media/khoa-ys/Personal/Materials/Master's Thesis/image_similarity_assessment/models/cable_siamese_wide_resnet_subclass_sampling.pth


100%|██████████| 149/149 [11:58<00:00,  4.82s/it]


----------------------------------- capsule -----------------------------------
No model for this backbone
----------------------------------- carpet -----------------------------------
No model for this backbone
----------------------------------- grid -----------------------------------
No model for this backbone
----------------------------------- hazelnut -----------------------------------
No model for this backbone
----------------------------------- leather -----------------------------------
No model for this backbone
----------------------------------- metal_nut -----------------------------------
No model for this backbone
----------------------------------- pill -----------------------------------
No model for this backbone
----------------------------------- screw -----------------------------------
No model for this backbone
----------------------------------- tile -----------------------------------
No model for this backbone
----------------------------------- toothbrush

100%|██████████| 109/109 [02:19<00:00,  1.28s/it]


----------------------------------- cable -----------------------------------
No model for this backbone
----------------------------------- capsule -----------------------------------
No model for this backbone
----------------------------------- carpet -----------------------------------
No model for this backbone
----------------------------------- grid -----------------------------------
No model for this backbone
----------------------------------- hazelnut -----------------------------------
No model for this backbone
----------------------------------- leather -----------------------------------
No model for this backbone
----------------------------------- metal_nut -----------------------------------
No model for this backbone
----------------------------------- pill -----------------------------------
No model for this backbone
----------------------------------- screw -----------------------------------
No model for this backbone
----------------------------------- tile ----